In [50]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import re

In [121]:
df = pd.DataFrame()

URLS_1 = ['https://www.exchange-rates.org/converter/usd-eur', 'https://www.exchange-rates.org/converter/eur-usd', 
         'https://www.exchange-rates.org/converter/uah-usd', 'https://www.exchange-rates.org/converter/usd-uah', 
         'https://www.exchange-rates.org/converter/eur-uah', 'https://www.exchange-rates.org/converter/uah-eur']

for URL_1 in URLS_1:
    
    from_cur = URL_1[-7:].split('-')[0].upper()
    to_cur = URL_1[-7:].split('-')[1].upper()
    
    req_1 = requests.get(URL_1)
    soup = BS(req_1.text, "html.parser")

    orig_path = "https://www.exchange-rates.org"

    urls = [orig_path + i[:-6] for i in re.findall(r'/\S*\d', str(soup.find_all('ul', class_='rates-by-year')[0]))]

    df_for_one = pd.DataFrame()

    for url in urls:
        req_2 = requests.get(url)
        soup_2 = BS(req_2.text, "html.parser")
        dates = [pd.to_datetime(date.text) for date in soup_2.find_all('a', class_='n')]
        values = [float(val.text.split()[-2]) for val in soup_2.find_all('span', class_='n')]
        _ = pd.DataFrame(values, index=dates, columns=[f'1_{from_cur}_to_{to_cur}'])
        df_for_one = pd.concat([df_for_one,_], axis=0)
    
    df_for_one = df_for_one.sort_index()
    display(df_for_one.tail())
        
    df = pd.concat([df, df_for_one], axis=1)

,1_USD_to_EUR
2024-04-10,0.9309
2024-04-11,0.9321
2024-04-12,0.9383
2024-04-15,0.9410
2024-04-16,0.9419


,1_EUR_to_USD
2024-04-10,1.0743
2024-04-11,1.0728
2024-04-12,1.0658
2024-04-15,1.0627
2024-04-16,1.0616


,1_UAH_to_USD
2024-04-10,0.02567
2024-04-11,0.02550
2024-04-12,0.02550
2024-04-15,0.02528
2024-04-16,0.02528


,1_USD_to_UAH
2024-04-10,38.957
2024-04-11,39.213
2024-04-12,39.223
2024-04-15,39.558
2024-04-16,39.558


,1_EUR_to_UAH
2024-04-10,41.850
2024-04-11,42.069
2024-04-12,41.805
2024-04-15,42.038
2024-04-16,41.997


,1_UAH_to_EUR
2024-04-10,0.02389
2024-04-11,0.02377
2024-04-12,0.02392
2024-04-15,0.02379
2024-04-16,0.02381


In [123]:
df = df.dropna().sort_index()
df

,1_USD_to_EUR,1_EUR_to_USD,1_UAH_to_USD,1_USD_to_UAH,1_EUR_to_UAH,1_UAH_to_EUR
2015-01-01,0.8265,1.2098,0.06322,15.819,19.138,0.05225
2015-01-02,0.8324,1.2014,0.06322,15.819,19.004,0.05262
2015-01-03,0.8335,1.1997,0.06322,15.819,18.978,0.05269
2015-01-04,0.8370,1.1946,0.06265,15.962,18.900,0.05243
2015-01-05,0.8391,1.1918,0.06322,15.818,18.852,0.05305
...,...,...,...,...,...,...
2024-04-10,0.9309,1.0743,0.02567,38.957,41.850,0.02389
2024-04-11,0.9321,1.0728,0.02550,39.213,42.069,0.02377
2024-04-12,0.9383,1.0658,0.02550,39.223,41.805,0.02392
2024-04-15,0.9410,1.0627,0.02528,39.558,42.038,0.02379


In [124]:
# df.to_csv('/diploma_info/datalake/currency_rates.csv')